<a href="https://colab.research.google.com/github/JuanJoseMV/neuraltextgen/blob/main/texygen_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !git clone https://github.com/geek-ai/Texygen.git
!git clone --recursive https://github.com/JuanJoseMV/neuraltextgen.git
!pip install -r /content/neuraltextgen/texygen/requirements.txt

fatal: destination path 'neuraltextgen' already exists and is not an empty directory.


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import sys, os
import os
os.chdir("/content/neuraltextgen/texygen")


In [6]:
from Texygen.utils.metrics.Bleu import Bleu

In [7]:
wiki = "In Siberia,Rasputin, un conducente di slitte ubriacone e analfabeta, viene a sapere mentre si trova all'osteria della morte della moglie di un ecclesiastico del posto, tale Meliti.\n" \
        " Il giorno seguente, Meliti, che ignora di essere rimasto vedovo, assume Rasputin che gli racconta di essere un veggente e gli annuncia la morte della moglie.\n" \
        " Quando Meliti scopre che questa è veramente morta, cade ai piedi di Rasputin, dichiarando la sua fede nelle premonizioni di quel sant'uomo.\n"\
        "Ben presto Rasputin stesso comincia a credere nei suoi poteri e si mette a predicare nelle campagne.\n"\
        "Nei suoi giri, incontra Anna, una giovane zingara che cade vittima dei suoi poteri ipnotici.\n"\
        "Il ciarlatano viene arrestato per furto, ma viene liberato dal principe Felix che, dopo avere sentito le voci che circolano sulle sue abilità profetiche di veggente, si reca al villaggio portandolo con sé a San Pietroburgo.\n"\
        "La predizione che Rasputin fa sulla nascita di un figlio maschio erede al trono si avvera e la sua influenza a corte aumenta, tanto che gli viene assegnato un palazzo che diventa la sua dimora.\n"


bert = 'in 2006 , the army announced plans to overhaul its schedule , notably reducing the march - october period , and shifting away from summer and winter exercises , which were annual , and the february - april period .\n'\
        'fuck , fuck , i kept it in that - " fuck , fuck , fuck , fuck , fuck . " * * * home , i mean , the house was a large part of my life .\n'\
        'my " father " ( matthew 15 : 17 - 26 ) \' s ornithological name ( " e " or " a " ) ; my " mother " , or my " sister " ;\n'\
        'london : macmillan . 1 june . thompson , thomas ( rev . 2003 ) . toronto ccp . volume 1 : the municipal elections in greater toronto , monday , 5 november 1962 . official party site .\n'\
        'b : b ( a + 1 ) = 3 and b : b ( a ) = = 2 c : a + ab c = 3 and are the values of all these variables defined as and .\n'\
        'a : barcelona , 1978 . el gimpy ( poet ) a : barcelona , 1982 . his books published posthumously include : la casa de historia catalan ( house of catalan history ) , impr .\n'

with open("wiki.txt", 'w') as f:
    f.write(wiki)

with open("bert.txt", 'w') as f:
    f.write(bert)

bleu_score = Bleu("bert.txt", "wiki.txt", 3)
bleu_score.get_score()

0.009386014712809162